# import

In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pickle
from pyvino.detector import DetectorHumanPose, DetectorBody
from actrec.util import moving_average, flatten_data, generate_col_names

/Users/yuya/anaconda3/envs/actrec/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


 # path

In [3]:
movie_files = ['forehand.MOV', 'backhand.MOV', 'foreslice.MOV', 'backslice.MOV', 'test.MOV']
# movie_files = ['backslice.mp4']

In [4]:
path_movie_dir = '../data/movie/fps30_mask/'
path_movie_test_file = '../data/movie/fps30_mask/backslice.mp4'

In [5]:
notebook_name = os.path.basename(os.getcwd())
timestamp = notebook_name.split('_')[1]
timestamp

'07'

In [6]:
path_generated_movie_dir = os.path.join('../data/07_GenerateMovie/', timestamp)
if not os.path.exists(path_generated_movie_dir):
    os.makedirs(path_generated_movie_dir)
path_save_file = os.path.join(path_generated_movie_dir, 'test_pred.mp4')

In [7]:
path_label_name_file = '../data/labels/labels_num.csv'
path_action_dir = os.path.join('../data/01_DataProcessing/', timestamp)
path_point_dir = os.path.join('../data/02_GetPoints/', timestamp)
path_split_dir = os.path.join('../data/04_Split/', timestamp)
path_test_file_dir = os.path.join(path_split_dir, 'test')
path_model_dir = os.path.join('../data/05_train/', timestamp)
path_model_file = os.path.join(path_model_dir, 'model.pkl')
path_pred_dir = os.path.join('../data/06_Evaluation/', timestamp)

In [8]:
file_names = ['forehand', 'backhand', 'foreslice', 'backslice', 'test']
# file_names = ['backslice']
movie_names = [file_name + '.mp4' for file_name in file_names]
points_file_names = [file_name + '.csv' for file_name in file_names]

In [9]:
objective_rev_dict = pd.read_csv(path_label_name_file).to_dict()['action_gt_name']

In [10]:
objective_rev_dict

{0: 'idle', 1: 'forehand', 2: 'backhand', 3: 'foreslice', 4: 'backslice'}

# model

In [11]:
det_b = DetectorBody()
det_h = DetectorHumanPose()

2019-09-16 14:20:26,506 - pyvino.detector.detector - INFO - person-detection-retail-0013 on Darwin
2019-09-16 14:20:26,507 - pyvino.detector.detector - INFO - The path to cpu_extension is /opt/intel/openvino/inference_engine/lib/intel64/libcpu_extension.dylib
2019-09-16 14:20:26,937 - pyvino.detector.detector - INFO - human-pose-estimation-0001 on Darwin
2019-09-16 14:20:26,938 - pyvino.detector.detector - INFO - The path to cpu_extension is /opt/intel/openvino/inference_engine/lib/intel64/libcpu_extension.dylib
2019-09-16 14:20:27,204 - pyvino.detector.detector - INFO - person-detection-retail-0013 on Darwin
2019-09-16 14:20:27,205 - pyvino.detector.detector - INFO - The path to cpu_extension is /opt/intel/openvino/inference_engine/lib/intel64/libcpu_extension.dylib


# data

# parameters

In [12]:
frame_length = 15

# process

## moving average

In [13]:
target_columns = ['RShoulder_x', 'RShoulder_y', 'RElbow_x', 'RElbow_y', 'RWrist_x', 'RWrist_y', 
                         'LShoulder_x', 'LShoulder_y', 'LElbow_x', 'LElbow_y', 'LWrist_x', 'LWrist_y']

## モデル読み込み

In [14]:
gbm = pickle.load(open(path_model_file, "rb"))
path_model_file

'../data/05_train/07/model.pkl'

In [15]:
feature_name_list = gbm.feature_name()

# 予測

In [16]:
mat = np.zeros(len(target_columns))
mat[:] = np.nan

In [57]:
num = 0
file_name = file_names[num]
movie_name = movie_names[num]
path_movie_test_file = os.path.join(path_movie_dir, movie_name)
path_movie_test_file

'../data/movie/fps30_mask/forehand.mp4'

In [58]:
path_test_file = os.path.join(path_split_dir, 'test', file_name + '.csv')
path_test_file

'../data/04_Split/07/test/forehand.csv'

In [59]:
objective = 'action_gt_num'
index_col = 'frame_num'

In [60]:
path_test_file

'../data/04_Split/07/test/forehand.csv'

In [61]:
test_df = pd.read_csv(path_test_file, index_col=index_col).head()
frame_num = min(test_df.index)

- frame
- pred norm pose and generate frame
- preproce
- pred action
- draw result

In [62]:
cap = cv2.VideoCapture(path_movie_test_file)
# 幅
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# 高さ
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# 総フレーム数
count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# fps
fps = int(cap.get(cv2.CAP_PROP_FPS))

canvas_org = np.zeros((height, width, 3), np.uint8)

In [63]:
features = []
for feature_name in feature_name_list:
    feature = feature_name.split('_')[0]
    features.append(feature)
features = list(np.unique(features))

In [64]:
path_test_file

'../data/04_Split/07/test/forehand.csv'

In [65]:
test_df = pd.read_csv(path_test_file, index_col=index_col).head()
frame_num = min(test_df.index)
frame_num

4044

target_columns = []
for feature in features:
    target_columns.append(feature + '_x')
    target_columns.append(feature + '_y')
features_frame = generate_col_names(target_columns)
len(features_frame)

In [26]:
def queue(arr, arr_add):
    dst = np.vstack([arr, arr_add])
    dst = np.delete(dst, obj=0, axis=0)
    return dst

In [27]:
path_movie_test_file

'../data/movie/fps30_mask/test.mp4'

In [28]:
color_codes = {'red' : (0,0,255),
                           'green' : (0,255,0),
                           'blue' : (255, 0,0),
                           'orange' : (80,127,255),
                           'skyblue' : (222,189,137)}

action_color = {'idle': 'green', 'forehand': 'red', 'backhand': 'blue', 'foreslice': 'orange', 'backslice': 'skyblue'}

In [29]:
movie_file_name = '{}.mp4'.format(file_name)
path_save_movie = os.path.join(path_generated_movie_dir, movie_file_name)

In [30]:
path_save_movie

'../data/07_GenerateMovie/07/test.mp4'

In [31]:
path_movie_test_file

'../data/movie/fps30_mask/test.mp4'

In [33]:
path_movie_test_file

'../data/movie/fps30_mask/test.mp4'

In [34]:
show=False
width_movie = 1280
height_movie = 720
fourcc = cv2.VideoWriter_fourcc(*"H264")
video = cv2.VideoWriter(path_save_movie, fourcc, fps, (width_movie, height_movie))

# checker_df  = pd.DataFrame([], columns=feature_actions_df.columns)

# test_df = pd.read_csv(path_test_file, index_col=index_col).head()
# frame_num = min(test_df.index)
frame_num = 0

window = 12
min_periods = 1

canvas_org = np.zeros((height, width, 3), np.uint8)

cap = cv2.VideoCapture(path_movie_test_file)

points_multi = np.empty((18, 2))
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
points_flattens = np.zeros([30,len(target_columns)])
points_flattens[:,:] = np.nan

poss = np.empty((0, 4))

pred = False

a = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_num % 100 == 0:
        print(frame_num)
    
    frame = np.array(frame)
    # frame = cv2.resize(frame, (width_movie, height_movie))

    ############################
    results = det_h.compute(frame, frame_flag=True, pred_flag=True, normalize_flag=True)
    frame = results['frame']
    ############################

    if len(results['preds']) == 0:
        points_flattens = np.vstack([points_flattens, mat])
        frame_num += 1
        video.write(frame)
        pred = False
        continue

    ############################
    points_dets = results['preds']
    new_area = 0
    largets_bbox_num = 0
    for num_det, points_det in points_dets.items():
        xmin, ymin, xmax, ymax = points_det['bbox']
        area = (ymax - ymin) * (xmax - xmin)
        if area > new_area:
            new_area = area
            largets_bbox_num = num_det
    ############################
    
    points = results['preds'][largets_bbox_num]['norm_points']
    points = pd.DataFrame(points, index=det_h.BODY_PARTS, columns=['x', 'y'])
    points = points.loc[features]
    
    point_cols = []
    for point in points.index:
        point_col = [point + '_x', point + '_y']
        point_cols.extend(point_col)
    
    points_flatten = points.values.flatten()
    points_flattens = queue(points_flattens, points_flatten)
    points_df = pd.DataFrame(points_flattens, columns=point_cols)
    # points_df = points_df.loc[:, target_columns]
    #  flatten
        
    ######
    # moving_average
    
    # frames to one columns
    #data_df = points_df.values.flatten()
    
    feature_actions_df = flatten_data(points_df)
    # sort features for model　input
    feature_actions_df = feature_actions_df.loc[:, feature_name_list]
    
    #checker_df = pd.concat([checker_df, feature_actions_df], axis=0)
    
    #  print(feature_actions_df)
    y_pred = gbm.predict(feature_actions_df, num_iteration=gbm.best_iteration)
    y_pred = np.argmax(y_pred, axis=1)[0]
    action_pred = objective_rev_dict[y_pred]
    pred = True

    ######    
    
    frame_num += 1
            
    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    if pred:
        action_color_code = color_codes[action_color[action_pred]]
        cv2.putText(frame, 'pred: ' + action_pred,
                                (int(xmax), int(ymin - 10)),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,0), 3, cv2.LINE_AA)
    
        cv2.putText(frame, 'pred: ' + action_pred,
                                    (int(xmax), int(ymin - 10)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, action_color_code, 2, cv2.LINE_AA)

    video.write(frame)
    if show:
        cv2.imshow('demo', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

video.release()

0


/Users/yuya/anaconda3/envs/actrec/lib/python3.6/site-packages/pyvino-0.0.1-py3.6.egg/pyvino/detector/detector.py:533: RuntimeWarning: invalid value encountered in greater
/Users/yuya/anaconda3/envs/actrec/lib/python3.6/site-packages/pyvino-0.0.1-py3.6.egg/pyvino/detector/detector.py:535: RuntimeWarning: invalid value encountered in greater


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
